# Time-optimal path planning
This notebook handles standard code for the computation of time-optimal paths in windfields

In [1]:
import os
import sys
import time
import numpy as np
from math import atan, cos, sin
sys.path.extend(['/home/bastien/Documents/work/mermoz', 
                 '/home/bastien/Documents/work/mermoz/src',
                 '/home/bastien/Documents/work/mdisplay',
                 '/home/bastien/Documents/work/mdisplay/src'])

from mermoz.feedback import TargetFB
from mermoz.mdf_manager import MDFmanager
from mermoz.params_summary import ParamsSummary
from mermoz.problem import MermozProblem
from mermoz.model import ZermeloGeneralModel
from mermoz.solver import Solver
from mermoz.stoppingcond import TimedSC, DistanceSC
from mermoz.wind import DiscreteWind
from mermoz.misc import *
from mdisplay.geodata import GeoData

from solver_utils import select_wind

### Problem definition

Initial point, target, UAV airspeed

In [2]:
# Scale factor
sf = 1e6

# Initial point
x_init = sf * np.array((0., 0.))

# Target
x_target = sf * np.array((1., 0.))

# Windfield boundaries
bl = sf * np.array((-0.1, -1.))
tr = sf * np.array((1.1, 1.))

# UAV airspeed in m/s
v_a = 23.

nx_rft = 101
ny_rft = 101

case_name = 'rad_gauss_1'

Set up the problem

In [3]:
# Prepare output directory
output_dir = f'/home/bastien/Documents/work/mermoz/output/example_solver_{case_name}'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Create a file manager to dump problem data
mdfm = MDFmanager()
mdfm.set_output_dir(output_dir)

# Problem coordinates
# coords = COORD_GCS
coords = COORD_CARTESIAN

# Problem wind data
from mermoz.wind import RadialGaussWind
total_wind = RadialGaussWind(sf * 0.5, sf * 0.05, sf * 0.2, 1/3 * np.log(0.5), v_a * 1.1)
def domain(x):
    return bl[0] < x[0] < tr[0] and bl[1] < x[1] < tr[1]

# Time window upper bound
# Estimated through great circle distance + 20 percent
T = 30. * 3600 # 1.2 * geodesic_distance(x_init, x_target, mode='rad') / v_a

# Creates the cinematic model
zermelo_model = ZermeloGeneralModel(v_a, coords=coords)
zermelo_model.update_wind(total_wind)

# Creates the navigation problem on top of the previous model
mp = MermozProblem(zermelo_model, T=T, coords=coords, autodomain=False, domain=domain)

### Solver parameters

In [4]:
auto_psi = 0.
psi_min = - DEG_TO_RAD * 90. #auto_psi - DEG_TO_RAD * 20.
psi_max = DEG_TO_RAD * 0. #auto_psi + DEG_TO_RAD * 20.

nt_pmp = 1000
opti_ceil = 0.01 * sf
neighb_ceil = opti_ceil/2.

solver = Solver(mp,
                x_init,
                x_target,
                T,
                psi_min,
                psi_max,
                output_dir,
                N_disc_init=2,
                opti_ceil=opti_ceil,
                neighb_ceil=neighb_ceil,
                n_min_opti=1,
                adaptive_int_step=False,
                N_iter=nt_pmp)

solver.log_config()
solver.setup()

Solve problem

In [5]:
t_start = time.time()
solver.solve_fancy()
t_end = time.time()
time_pmp = t_end - t_start

Shooting -90.0
Shooting 0.0
857675.94238, -45.00
It   1 - Shoot -45.0 - Depth   1 - Crit 1001685.8, 100% - Branching LU
It   2 - Shoot -22.5 - Depth   2 - Crit 644777.4, 64% - Branching  U
It   3 - Shoot -11.2 - Depth   3 - Crit 694918.8, 69% - Branching LU
It   4 - Shoot -16.9 - Depth   4 - Crit 676283.1, 67% - Branching LU
It   5 - Shoot -19.7 - Depth   5 - Crit 662898.2, 66% - Branching LU
It   6 - Shoot -21.1 - Depth   6 - Crit 654591.4, 65% - Branching LU
It   7 - Shoot -21.8 - Depth   7 - Crit 649900.8, 64% - Branching LU
It   8 - Shoot -22.1 - Depth   8 - Crit 647397.4, 64% - No branching
It   9 - Shoot -21.4 - Depth   8 - Crit 652296.2, 65% - No branching
It  10 - Shoot -20.4 - Depth   7 - Crit 658908.1, 65% - Branching LU
It  11 - Shoot -20.7 - Depth   8 - Crit 656793.3, 65% - No branching
It  12 - Shoot -20.0 - Depth   8 - Crit 660941.4, 66% - No branching
It  13 - Shoot -18.3 - Depth   6 - Crit 670048.5, 67% - Branching LU
It  14 - Shoot -19.0 - Depth   7 - Crit 666600.9, 66

### Explicit control laws

In [6]:
sc = DistanceSC(lambda x: np.linalg.norm(x - x_target), opti_ceil)
sc = TimedSC(T)
from mermoz.feedback import FixedHeadingFB, ConstantFB

mp.load_feedback(FixedHeadingFB(mp.model.wind, v_a, 0., mp.coords))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

mp.load_feedback(ConstantFB(0.))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

### RFT

In [6]:
from mermoz.rft import RFT
t_start = time.time()
rft = RFT(bl, tr, T, nx_rft, ny_rft, mp, x_init, kernel='matlab', coords=COORD_CARTESIAN)
rft.compute()
t_end = time.time()
time_rft = t_end - t_start


tol =

       12000

	151 steps in 3.68 seconds from 0 to 13500
	151 steps in 4 seconds from 13500 to 27000
	151 steps in 2.67 seconds from 27000 to 40500
	151 steps in 3.97 seconds from 40500 to 54000
	151 steps in 2.83 seconds from 54000 to 67500
	151 steps in 2.76 seconds from 67500 to 81000
	151 steps in 3 seconds from 81000 to 94500
	151 steps in 2.65 seconds from 94500 to 108000

Total execution time 26.08 seconds


Dump everything to output directory

In [7]:
nx = 1001
ny = 1001
mdfm.dump_wind(total_wind, nx=101, ny=101, bl=bl, tr=tr)
mdfm.dump_trajs(mp.trajs)
# rft.dump_rff(output_dir)


ps = ParamsSummary({}, output_dir)
ps.load_from_solver(solver)
ps.add_param('bl_wind', tuple(bl))
ps.add_param('tr_wind', tuple(tr))
#ps.add_param('nx_wind', nx)
#ps.add_param('ny_wind', ny)
ps.add_param('nt_pmp', nt_pmp)
try:
    ps.add_param('pmp_time', time_pmp)
except NameError:
    pass
ps.dump()